In [14]:
import os
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import numpy as np

In [15]:
# Paths
dataset_path = "Dataset1"
output_path = "features_dataset1.npy"  # Path to save extracted features

# Pre-trained model and processor
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-11): 12 x Dinov2Layer(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2SdpaAttention(
          (attention): Dinov2SdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_pa

In [16]:
# Preprocessing pipeline
preprocess = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# Feature extraction
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [17]:
# Iterate through dataset and extract features
features = []
labels = []
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(os.listdir(dataset_path))}

for class_name, class_idx in class_to_idx.items():
    class_path = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        try:
            feature_vector = extract_features(img_path)
            features.append(feature_vector)
            labels.append(class_idx)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

In [18]:
# Save features and labels
features = np.array(features)
labels = np.array(labels)
np.save(output_path, {"features": features, "labels": labels})

In [20]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load extracted features
data = np.load("features_dataset1.npy", allow_pickle=True).item()
features, labels = data["features"], data["labels"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train an SVM classifier
svm_clf = SVC(kernel='linear', C=1.0, random_state=42)  # Linear kernel; adjust C for regularization
svm_clf.fit(X_train, y_train)

# Make predictions
predictions = svm_clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names=['Laptops', 'Smartphones']))

Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

     Laptops       1.00      1.00      1.00         5
 Smartphones       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8

